## agentを利用したアプリケーション

### 準備

必要なライブラリのインストール

In [1]:
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


環境変数の読み込み

In [2]:
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [ ]:

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_cohere import CohereEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, CohereEmbeddings(model="embed-multilingual-v3.0"))
retriever = vector.as_retriever()

retriever.invoke("how to upload a dataset")[0]

### Tool

In [10]:
from langchain_core.tools import tool
from enum import Enum

# 曜日のEnum定義
class DayOfWeek(Enum):
    MONDAY = 'Monday'
    TUESDAY = 'Tuesday'
    WEDNESDAY = 'Wednesday'
    THURSDAY = 'Thursday'
    FRIDAY = 'Friday'
    SATURDAY = 'Saturday'
    SUNDAY = 'Sunday'

day_values_map = {
    DayOfWeek.MONDAY: 0,
    DayOfWeek.TUESDAY: 10,
    DayOfWeek.WEDNESDAY: 20,
    DayOfWeek.THURSDAY: 30,
    DayOfWeek.FRIDAY: 40,
    DayOfWeek.SATURDAY: 50,
    DayOfWeek.SUNDAY: 100,
}

@tool
def combat_power(day: DayOfWeek) -> int:
    """
    曜日に対応するBigBaByの戦闘力を計算します。

    Args:
      day: 曜日です。
    """

    return day_values_map.get(day)

print(combat_power.name)
print(combat_power.description)
print(combat_power.args)


combat_power
曜日に対応するBigBaByの戦闘力を計算します。

Args:
  day: 曜日です。
{'day': {'$ref': '#/$defs/DayOfWeek'}}


In [11]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)
tools = [combat_power]#, retriever_tool]
tools

[StructuredTool(name='combat_power', description='曜日に対応するBigBaByの戦闘力を計算します。\n\nArgs:\n  day: 曜日です。', args_schema=<class 'langchain_core.utils.pydantic.combat_power'>, func=<function combat_power at 0x7f91ed9d0720>)]

In [13]:
from langchain_cohere import ChatCohere

model = ChatCohere(model="command-r")
model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke("おはよう")
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: おはようございます！今日も一日頑張りましょう。
ToolCalls: []


In [14]:
response = model_with_tools.invoke("BigBaByの月曜の次の日の戦闘力は？")
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: I will ask for the combat power on Monday, and then add one day to it.
ToolCalls: [{'name': 'combat_power', 'args': {'day': 'Monday'}, 'id': 'c149b7ca98c24a41a2c5c829fce4b1b1', 'type': 'tool_call'}]


In [23]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver  # an in-memory checkpointer

system_message = "あなたは有用なアシスタントです。"
memory = MemorySaver()
langgraph_agent_executor = create_react_agent(
  model, tools,state_modifier=system_message, checkpointer=memory
)
config = {"configurable": {"thread_id": "test-thread"}}
langgraph_agent_executor.invoke(
  {
    "messages": [
      ("user","BigBaByの木曜の戦闘力は？"),
    ]
  },
  config,
)

{'messages': [HumanMessage(content='BigBaByの木曜の戦闘力は？', additional_kwargs={}, response_metadata={}, id='45d1d4d7-cef6-47c2-b4be-16281506c080'),
  AIMessage(content='I will use the provided function to find out the combat power for Thursday and relay this information to the user.', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '1065eb17-11ff-42dc-9b79-05c71f426b45', 'tool_calls': [{'id': '2ac2ba7166a9429ca1522372bccf0c6f', 'function': {'name': 'combat_power', 'arguments': '{"day": "Thursday"}'}, 'type': 'function'}], 'token_count': {'input_tokens': 925.0, 'output_tokens': 64.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '1065eb17-11ff-42dc-9b79-05c71f426b45', 'tool_calls': [{'id': '2ac2ba7166a9429ca1522372bccf0c6f', 'function': {'name': 'combat_power', 'arguments': '{"day": "Thu